 # FaceBase Anatomical Term Curation Report

## Executive Summary

This report details the implementation and evaluation of an automated anatomical term curation pipeline for FaceBase data. Through a multi-strategy approach combining embedding technologies with large language models (LLMs), we successfully achieved high-confidence anatomical term curation for up to 99% of previously uncurated terms, improving data standardization and research interoperability.

## Background: FaceBase

FaceBase serves as a centralized data repository for craniofacial research, facilitating collaboration and accelerating discoveries in craniofacial development and disorders. A significant challenge in maintaining biomedical databases like FaceBase is ensuring consistent terminology across research contributions.

## The Curation Challenge

Anatomical term curation involves more than simple standardization. It requires:

1. Interpreting researcher-supplied terminology in proper biological context
2. Mapping diverse descriptive terms to established ontological standards
3. Resolving ambiguities where multiple interpretations are possible
4. Ensuring consistency across species, developmental stages, and experimental contexts

Manual curation by domain experts is the gold standard but is time-intensive and difficult to scale. Our pipeline aims to automate this process while maintaining expert-level quality.

## Our Dataset

The input dataset consisted of 100 anatomical terms from FaceBase with their corresponding uncurated variations. This dataset was specifically designed to test our curation pipeline's ability to handle realistic anatomical terminology variations:

### Dataset Structure
Each row in our dataset contains:
- **Standard FaceBase anatomical term**: The official standard term from FaceBase anatomy vocabulary (e.g., "maxillary palatine suture")
- **Uncurated terminology**: A variation of the standard term following natural variation patterns (e.g., "the maxillary palatine suture of the developing specimen")
- **Rich contextual metadata**:
  - Dataset information (ID, title, description)
  - Biological context (species, developmental stage, stage description)
  - Specimen and genotype information

### Uncurated Term Generation
The uncurated terms were systematically generated to represent the most common variation patterns found in biomedical literature, distributed according to research-based frequencies:

1. **Synonyms** (25-30%): Alternative terms referring to the same anatomical structure
   - Example: "mandible" → "lower jaw"

2. **Word Order/Format Variations** (15-20%): Rearranged word order while maintaining meaning
   - Example: "maxillary palatine suture" → "palatine maxillary suture"

3. **Alternative Terminology** (15-20%): Discipline-specific jargon or technical alternatives
   - Example: "suture" → "articulation"

4. **Generalization/Under-Specification** (15-20%): Broader, less specific terms
   - Example: "molar tooth" instead of "upper jaw molar"

5. **Excessive Verbosity** (10-15%): Overly descriptive phrasing
   - Example: "frontal bone" → "the frontal bone of the developing specimen"

6. **Over-Specification** (5-10%): Unnecessarily detailed terms with redundant modifiers
   - Example: "maxilla" → "upper adult mammalian maxilla"

### Validation Plan
To ensure these variations accurately represent the types of terminology inconsistencies found in real scientific literature, we plan to validate them through expert review. Biomedical specialists will assess each uncurated term for naturalness—whether it realistically resembles how a researcher might refer to that anatomy in practice—and suggest improvements while maintaining the same variation type.

This carefully constructed dataset allows us to systematically evaluate our curation pipeline's performance across different types of terminology variations while providing the biological context necessary for accurate term mapping.

## Methodology

Our pipeline integrates computational methods in a layered approach:

### Foundation Layer
1. **Standard Term Embedding**
   - Created vector representations of the standard FaceBase anatomical terms
   - Utilized scientific BERT models (specifically "gsarti/scibert-nli") for biomedical embeddings
   - Established a semantic space of standardized terminology

2. **Embedding-based Similarity Analysis**
   - Computed semantic similarity between uncurated terms and standard FaceBase anatomical terms
   - Identified top candidate matches based on semantic meaning
   - Created a foundation for context-enriched curation approaches

### Data Context Integration
We leverage the structured metadata available in each row of the FaceBase dataset, including:
- Species information (e.g., "Mus musculus")
- Developmental stage descriptions (e.g., "Mouse embryonic stage E16.5")
- Dataset descriptions containing experimental context
- Specimen and genotype information

This row-specific structured metadata provides the unique biological context for each term being curated, allowing the LLM to make more informed decisions based on the specific experimental setting of each anatomical term.

### Conceptual Framework and RAG Considerations

The conceptual foundation of our approach draws inspiration from Retrieval-Augmented Generation (RAG) systems while addressing their inherent limitations in the context of heterogeneous scientific databases. RAG systems typically enhance LLM performance by retrieving relevant information from large knowledge bases through semantic similarity before generating responses.

#### Limitations of Traditional RAG for FaceBase

FaceBase (https://www.facebase.org/) presents unique challenges that render traditional RAG approaches suboptimal:

1. **Multi-source dataset heterogeneity**: FaceBase integrates numerous datasets from independent research groups, each using different terminological conventions, experimental designs, and anatomical references. Our analysis of the input data revealed multiple distinct source datasets, each with its own context and terminology patterns. A traditional RAG system would struggle to properly discriminate between these contexts when retrieving information.

2. **Context-dependent terminology interpretation**: The same anatomical term can have different interpretations depending on species, developmental stage, and experimental methodology—all of which vary considerably across FaceBase entries. For example, the term "palate" requires different interpretations in embryonic versus adult specimens, or across different species. Standard RAG approaches lack the fine-grained context awareness needed for accurate term disambiguation.

3. **Limited access to comprehensive textual resources**: In practical implementation scenarios, complete access to all relevant textual resources for RAG indexing is often unfeasible. Many research datasets contain restricted or sensitive information, and new datasets are continuously added to the platform. Our approach needed to function effectively without requiring extensive knowledge base construction for each new dataset addition.

4. **Specialized terminology hallucination risk**: The highly specialized nature of anatomical terminology increases the risk of hallucination in traditional RAG systems. When confronted with obscure or ambiguous anatomy terms, a general RAG approach might retrieve loosely related but inaccurate information from its knowledge base, potentially introducing errors rather than resolving them.

#### Adaptation of Semantic Similarity Principles

While traditional RAG implementation presented challenges for our specific use case, we recognized that the fundamental power of RAG systems lies in their use of semantic similarity through vector embeddings. Therefore, we extracted this core principle and implemented it in a simplified yet more targeted approach:

1. **Focused embedding of standard vocabulary**: Rather than embedding vast knowledge bases, we created precise vector representations specifically for the standard FaceBase anatomical terms. This created a bounded, domain-specific semantic space optimized for the curation task.

2. **Row-specific metadata utilization**: Instead of retrieving external knowledge, our approach leverages the structured metadata already present in each data row—providing the specific experimental context needed for accurate term interpretation without requiring extensive external knowledge bases.

3. **Two-stage hybrid architecture**: By implementing a confidence-based routing system between semantic similarity matching and full vocabulary consideration, we maintained the benefits of similarity-based term matching while adding a verification mechanism that improved overall accuracy.

This adaptation allowed us to harness the semantic matching power that makes RAG systems effective while overcoming the specific challenges presented by FaceBase's heterogeneous data environment. The resulting approach is more streamlined, doesn't require access to extensive external knowledge bases, and can operate effectively across diverse datasets without confusion between their different contexts.

### Curation Approaches
3. **Context-enriched Standard terms guided Curation**
   - Leveraged LLM knowledge to interpret uncurated terms
   - Provided complete standard FaceBase anatomy vocabulary as reference
   - Enriched each query with structured metadata from the same data row (species, stage, description)
   - Assessed model confidence in term mappings

4. **Context-enriched similarity-guided Curation**
   - Combined embedding-identified candidates with LLM reasoning
   - Incorporated row-specific structured metadata (biological context) for each term
   - Enabled the LLM to consider term semantics in the specific experimental context
   - Maintained high confidence threshold for quality assurance

5. **Hybrid Context-enriched Curation Strategies**
   - Two-directional hybrid approach:
     - Similarity → Standard: First attempt with similarity-guided, fallback to standard terms
     - Standard → Similarity: First attempt with standard terms, fallback to similarity-guided
   - Dynamic routing of low-confidence terms to the complementary approach
   - Each curation decision leverages the row-specific structured metadata
   - Confidence thresholding to ensure high-quality curation

## Results

| Approach | High-Confidence Curations | Percentage |
|----------|---------------------------|------------|
| Context-enriched similarity-guided Curation | 91/100 | 91% |
| Context-enriched Standard terms guided Curation | 86/100 | 86% |
| Hybrid Context-enriched Curation (Similarity → Standard) | 97/100 | 97% |
| Hybrid Context-enriched Curation (Standard → Similarity) | 99/100 | 99% |

## Significance

This pipeline represents a significant advancement in biomedical data curation:

1. **Quality Enhancement**: Transforms researcher-supplied terminology into standardized ontological terms

2. **Metadata-Informed Curation**: Leverages the structured metadata from each data row to inform curation decisions, ensuring biological context relevance

3. **Confidence Assessment**: Provides reliability metrics for each curation decision

4. **Efficiency**: Reduces manual curation workload while maintaining high quality standards

5. **Vocabulary Evolution**: Identifies potential new terms for inclusion in standard vocabulary

## Key Insights

Our experiments revealed several important insights:

1. **Row-specific metadata is crucial**: Incorporating structured metadata from each data row (species, stage description, dataset information) significantly improves curation accuracy across all approaches.

2. **Similarity-guided approach shows impressive standalone performance**: The Context-enriched similarity-guided Curation achieved 91% accuracy by itself, highlighting the effectiveness of combining semantic similarity with biological context.

3. **Two-stage approaches reach near-perfect accuracy**: Despite the strong performance of the similarity-guided approach, hybrid strategies further improved results to 97-99% accuracy by addressing edge cases where either approach alone might struggle.

4. **Directional preference matters**: The Standard → Similarity hybrid approach achieved the highest accuracy (99%), suggesting that attempting standard term matching first, then using similarity guidance for challenging cases, is optimal.

5. **Confidence thresholding enables effective triage**: Our confidence-based routing strategy successfully identified challenging terms (9-15% of cases) and directed them to the complementary approach for resolution.

6. **Structured knowledge guides LLM reasoning**: The consistent metadata structure, combined with semantic similarity candidates and standard FaceBase anatomical vocabulary, provides the LLM with structured knowledge that enables effective decision-making for accurate term curation.

## Next Steps

Based on our findings, we recommend the following next steps:

1. **Parameter optimization**: Fine-tune multiple system parameters including confidence thresholds, temperature, batch size, number of semantic candidates, and context character limits to maximize accuracy while minimizing API costs.

2. **Tiered efficiency strategy**: For large-scale datasets, implement a multi-tier approach:
   - First tier: Simple exact matching for terms already in the standard vocabulary
   - Second tier: Fuzzy matching algorithms to handle spelling variations and slight errors
   - Third tier: Full context-enriched curation for complex cases requiring sophisticated analysis
   This approach would significantly reduce API costs while maintaining high accuracy.

3. **Continuous learning with memory**: Establish a feedback loop where human curators can review and correct any remaining errors. Build a persistent knowledge base to track curator decisions and intuition patterns over time, allowing the system to learn from these corrections and gradually improve.

4. **Vocabulary expansion**: Analyze terms classified as "UNKNOWN" to identify potential candidates for inclusion in the standard FaceBase vocabulary.

## Conclusion

Our anatomical term curation pipeline successfully addresses a critical challenge in biomedical data management. The Context-enriched similarity-guided Curation approach alone achieves 91% accuracy, demonstrating the effectiveness of combining embedding-based similarity analysis with row-specific structured metadata to inform LLM reasoning.

Building upon this foundation, our two-stage hybrid strategies further enhance performance to 97-99% accuracy. Notably, the Standard → Similarity pathway (99% accuracy) and Similarity → Standard pathway (97% accuracy) both demonstrate statistically significant improvements over single-stage approaches. These results indicate that the integration of complementary methodologies yields optimal performance for anatomical term standardization.

Each approach leverages the specific experimental context available in the structured metadata of individual data rows, allowing for context-sensitive curation decisions. The integration of semantic similarity metrics with standard FaceBase anatomical vocabulary provides a robust framework for mapping terminology variations to standardized terms.

This system not only enhances the immediate interoperability of FaceBase data but also establishes a methodological foundation for ongoing terminology refinement and standardization across the craniofacial research community. The demonstrated accuracy rates of 91-99% across different methodological implementations suggest that automated curation can effectively replace manual processes while maintaining high-quality standardization.

In [ ]:
!pip install openai==0.28
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"

In [ ]:
# ╔══════════════════════════════════════════════════════════╗
# ║  FACEBASE ANATOMICAL TERM CURATION PIPELINE             ║
# ║  Two-Stage Hybrid Curation Strategy                     ║
# ╚══════════════════════════════════════════════════════════╝

import os
import json
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
from google.colab import files
import umap.umap_ as umap

# ====================== CONFIGURATION ======================
# API key and model settings
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]

# Model configuration
MODEL_NAME = "gpt-4o-mini"  # or "gpt-4o"
TEMPERATURE = 0.0
CONF_THRESHOLD = 0.80
BATCH_SIZE = 4

# Embedding model
EMBEDDING_MODEL = "gsarti/scibert-nli"

# Context settings for LLM prompting
CONTEXT_COLS = ["species", "stage_description", "dataset_description"]
CONTEXT_CHARS = 400
SHOW_SCORES = True

# File paths
STD_TERMS_CACHE = "standard_terms_embeddings.pkl"

# =================== UTILITY FUNCTIONS ====================
def robust_json_parse(text):
    """Parse JSON with fallback options for different response formats from the LLM"""
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        # Try to extract JSON block if wrapped in markdown
        start, end = text.find('['), text.rfind(']')
        if start != -1 and end != -1:
            try:
                return json.loads(text[start:end+1])
            except Exception:
                pass
        # Fallback: parse line by line
        objs = []
        for line in text.splitlines():
            line = line.strip()
            if not line:
                continue
            try:
                objs.append(json.loads(line))
            except json.JSONDecodeError:
                print("⚠️  Could not parse line:", line[:120])
        return objs

def process_in_batches(items, batch_size, process_func):
    """Process a list of items in batches with a processing function"""
    results = []
    for i in tqdm(range(0, len(items), batch_size)):
        batch = items[i:i+batch_size]
        batch_results = process_func(batch)
        results.extend(batch_results)
    return results

def format_context(row, context_cols, max_chars):
    """Format context information from specified columns"""
    ctx_parts = []
    for col in context_cols:
        if col in row and pd.notna(row[col]):
            # You can choose to give dataset_description more characters if needed
            col_max_chars = max_chars * 2 if col == "dataset_description" else max_chars
            ctx_parts.append(f"{col}: {str(row[col])[:col_max_chars]}")
    return " | ".join(ctx_parts) if ctx_parts else "—"

def calculate_accuracy(df, column_name):
    """Calculate accuracy based on confidence threshold"""
    confident_count = (df[f"confidence_{column_name}"] >= CONF_THRESHOLD).sum()
    total_count = len(df)
    accuracy = confident_count / total_count
    return accuracy, confident_count, total_count

# =============== EMBEDDING FUNCTIONALITY =================
def upload_input_dataset():
    """Upload and load the input dataset CSV file"""
    print("Please upload your CSV file containing the standard anatomical terms...")
    uploaded_input = files.upload()
    input_filename = next(iter(uploaded_input))
    input_df = pd.read_csv(input_filename)
    print(f"Loaded input file with {len(input_df)} rows.")
    return input_df

def extract_standard_terms(df, exclude_terms=['tail', 'face']):
    """Extract unique standard terms from the input dataset"""
    # Extract unique terms from anatomy column
    unique_terms = df['anatomy'].dropna().unique().tolist()
    print(f"Found {len(unique_terms)} unique terms before filtering.")

    # Filter out excluded terms
    filtered_terms = [term for term in unique_terms if term.lower() not in [t.lower() for t in exclude_terms]]
    print(f"Retained {len(filtered_terms)} standard terms after filtering out {exclude_terms}.")

    return filtered_terms

def create_embeddings(terms, model_name=EMBEDDING_MODEL, use_cached=True):
    """Create or load embeddings for a list of terms"""
    # Check if embeddings already exist
    cache_file = f"embeddings_{model_name.replace('/', '_')}.pkl"

    if os.path.exists(cache_file) and use_cached:
        print(f"Loading cached embeddings from {cache_file}")
        with open(cache_file, 'rb') as f:
            data = pickle.load(f)
            # Check if it's the new format or old format
            if isinstance(data, tuple) and len(data) == 2:
                embeddings, model = data
            else:
                # Old format - just embeddings
                embeddings = data
                from sentence_transformers import SentenceTransformer
                model = SentenceTransformer(model_name)
            return embeddings, model

    print(f"Creating embeddings using {model_name}...")

    # Load model
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer(model_name)

    # Generate embeddings
    embeddings = {}
    for term in tqdm(terms):
        embeddings[term] = model.encode(term)

    # Save embeddings for future use
    with open(cache_file, 'wb') as f:
        pickle.dump((embeddings, model), f)

    print(f"Created embeddings for {len(embeddings)} standard terms.")
    return embeddings, model

def visualize_embeddings(embeddings):
    """Visualize term embeddings in 2D using UMAP"""
    # Extract embeddings and terms
    terms = list(embeddings.keys())
    embedding_matrix = np.array([embeddings[term] for term in terms])

    # Reduce dimensionality for visualization
    reducer = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='cosine')
    embedding_2d = reducer.fit_transform(embedding_matrix)

    # Create a dataframe for plotting
    vis_df = pd.DataFrame({
        'term': terms,
        'x': embedding_2d[:, 0],
        'y': embedding_2d[:, 1],
        'word_count': [len(term.split()) for term in terms]
    })

    # Plot
    plt.figure(figsize=(12, 10))
    sns.scatterplot(data=vis_df, x='x', y='y', hue='word_count', size='word_count',
                    sizes=(20, 200), alpha=0.7, palette='viridis')

    # Add labels for points - but be selective to avoid overcrowding
    for idx, row in vis_df.sample(min(20, len(vis_df))).iterrows():
        plt.text(row['x']+0.1, row['y']+0.1, row['term'], fontsize=8)

    plt.title('Visualization of Standard Anatomical Term Embeddings')
    plt.savefig('standard_terms_embedding_visualization.png')
    plt.show()
    print("Saved visualization as 'standard_terms_embedding_visualization.png'")

    return vis_df

def load_or_create_standard_embeddings(input_df=None):
    """Load standard term embeddings from cache or create new ones"""
    if os.path.exists(STD_TERMS_CACHE):
        print(f"Loading standard terms from cache: {STD_TERMS_CACHE}")
        with open(STD_TERMS_CACHE, 'rb') as f:
            saved = pickle.load(f)
        standard_terms = saved["standard_terms"]
        standard_embeddings = saved["embeddings"]
        model = saved["model"]
        print(f"Loaded {len(standard_terms)} standard FaceBase terms.")
        return standard_terms, standard_embeddings, model

    # If no cached data, create new embeddings
    if input_df is None:
        input_df = upload_input_dataset()

    standard_terms = extract_standard_terms(input_df)
    print(f"\nExtracted {len(standard_terms)} standard terms:")
    for i, term in enumerate(standard_terms, 1):
        print(f"{i}. {term}")

    # Create embeddings for standard terms
    standard_embeddings, model = create_embeddings(standard_terms)

    # Visualize embeddings
    vis_df = visualize_embeddings(standard_embeddings)

    # Save standard terms and their embeddings
    output = {
        'standard_terms': standard_terms,
        'embeddings': standard_embeddings,
        'model': model
    }

    with open(STD_TERMS_CACHE, 'wb') as f:
        pickle.dump(output, f)

    print(f"\nEmbedded all standard terms and saved to '{STD_TERMS_CACHE}'")
    return standard_terms, standard_embeddings, model

def compute_top_n_matches(df, model, standard_terms, standard_embeddings, unc_col="anatomy_uncurated", top_n=3):
    """Compute top N most similar standard terms for each uncurated term"""

    def get_top_matches(term, n=top_n):
        emb = model.encode(term)
        sims = [(t, cosine_similarity([emb], [standard_embeddings[t]])[0][0])
                for t in standard_terms]
        sims.sort(key=lambda x: x[1], reverse=True)
        return sims[:n]

    # Add result columns
    for i in range(1, top_n+1):
        df[f"match_{i}"] = None
        df[f"score_{i}"] = None

    # Compute top matches for each row
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        term = row[unc_col]
        if pd.isna(term): continue
        matches = get_top_matches(term)

        # Print just a single example for the first few terms
        if idx < 5:
            print(f"Sample - Uncurated term: {term}")
            print(f"  Top match: {matches[0][0]} (similarity score: {matches[0][1]:.4f})")

        for i, (m, s) in enumerate(matches, 1):
            df.loc[idx, f"match_{i}"] = m
            df.loc[idx, f"score_{i}"] = s

    print(f"Computed top {top_n} matches for {len(df)} terms.")
    return df

# ================ LLM FUNCTIONALITY ===================
# System prompts
SYSTEM_PROMPT_STD_TERMS = """You are an expert biomedical curator.

Below is the entire FaceBase canonical anatomy vocabulary:

{vocab_block}

For each uncurated phrase you receive, return:
  → curated_term  = the SINGLE most appropriate canonical term
                    from the list above, OR "UNKNOWN"
  → confidence    = 0‑1 (1.0 = absolutely sure)

Respond **only** with a JSON list (same order)."""

SYSTEM_PROMPT_SIMILARITY = """You are an expert biomedical curator.

For each record you receive:
- an uncurated anatomy phrase
- biological context (species, stage, description)
- the three highest‑similarity candidates from the full FaceBase vocabulary

Your task is to *curate* the uncurated phrase—produce the single best
canonical FaceBase term you would assign as an expert curator. You may
choose one of the three candidates *or* write a different canonical term
if none is satisfactory. If no suitable FaceBase term exists, return
"UNKNOWN". Always include a confidence score (0‑1).

Return only a JSON list (same order) where each object has:
  "curated_term": <string>
  "confidence":   <float 0‑1>
"""

JSON_REMINDER = "Return ONLY the raw JSON list—no markdown, no commentary."

def llm_curate_batch(batch, system_prompt, is_similarity_guided=False, standard_terms=None, unc_col="anatomy_uncurated"):
    """Generic LLM curation function that supports different prompt styles"""

    if is_similarity_guided:
        # Context-enriched similarity-guided approach (with top matches)
        user_lines = []
        for i, r in enumerate(batch):
            ctx = format_context(r, CONTEXT_COLS, CONTEXT_CHARS)

            # Candidate list lines
            cand_lines = []
            for label, j in zip("ABC", [1, 2, 3]):
                m, s = r[f"match_{j}"], r[f"score_{j}"]
                sim = f"(sim={s:.3f})" if SHOW_SCORES else ""
                cand_lines.append(f"{label}) {m} {sim}".strip())

            line_prefix = "curate among"

            user_lines.append(
                f"{i}. uncurated: {r[unc_col]}\n"
                f"   context: {ctx}\n"
                f"   {line_prefix}:\n     " + "\n     ".join(cand_lines)
            )
    else:
        # Context-enriched Standard terms guided approach
        if standard_terms is None:
            raise ValueError("Standard terms must be provided for standard terms guided curation")

        # Format the vocabulary block
        vocab_block = "\n".join(f"- {t}" for t in standard_terms)
        system_prompt = system_prompt.format(vocab_block=vocab_block)

        user_lines = []
        for i, r in enumerate(batch):
            ctx = format_context(r, CONTEXT_COLS, CONTEXT_CHARS)
            user_lines.append(f"{i}. term: {r[unc_col]}\n   context: {ctx}")

    # DEBUG: Print the prompt for the first term
    if not hasattr(llm_curate_batch, 'approach_seen'):
        llm_curate_batch.approach_seen = set()

    approach_type = "similarity-guided" if is_similarity_guided else "standard-terms-guided"
    if approach_type not in llm_curate_batch.approach_seen:
        print(f"\n{'='*50}")
        print(f"PROMPT FOR FIRST TERM USING {approach_type.upper()} APPROACH")
        print(f"{'='*50}")
        print("SYSTEM PROMPT:")
        print(system_prompt[:1000] + ("..." if len(system_prompt) > 1000 else ""))
        print("\nUSER PROMPT (first term only):")
        if user_lines:
            print(user_lines[0])
        print(f"{'='*50}\n")
        llm_curate_batch.approach_seen.add(approach_type)

    # Call the OpenAI API
    resp = openai.ChatCompletion.create(
        model=MODEL_NAME,
        temperature=TEMPERATURE,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "\n\n".join(user_lines)},
            {"role": "user", "content": JSON_REMINDER}
        ]
    )
    response_text = resp.choices[0].message.content.strip()

    # Parse the JSON response
    return robust_json_parse(response_text)

# Initialize the static set to track which approaches we've seen
llm_curate_batch.approach_seen = set()

def llm_curate_dataset(df, approach_type, standard_terms=None):
    """Curate dataset using the specified approach and configuration"""
    records = df.to_dict("records")
    unc_col = "anatomy_uncurated"

    # Configure approach
    if approach_type == "standard_terms":
        if standard_terms is None:
            raise ValueError("Standard terms must be provided for standard terms guided curation")

        system_prompt = SYSTEM_PROMPT_STD_TERMS
        is_similarity_guided = False
        suffix = "std_terms"

    elif approach_type == "similarity":
        system_prompt = SYSTEM_PROMPT_SIMILARITY
        is_similarity_guided = True
        suffix = "similarity"

    else:
        raise ValueError(f"Unknown approach type: {approach_type}")

    # Process the dataset in batches
    def process_batch(batch):
        return llm_curate_batch(
            batch,
            system_prompt=system_prompt,
            is_similarity_guided=is_similarity_guided,
            standard_terms=standard_terms
        )

    results = process_in_batches(records, BATCH_SIZE, process_batch)

    # Extract results - Ensuring results match dataframe length
    curated_terms = []
    confidences = []

    for i, record in enumerate(records):
        if i < len(results):
            result = results[i]
            curated_terms.append(result.get("curated_term", "UNKNOWN"))
            confidences.append(float(result.get("confidence", 0)))
        else:
            # Handle any missing results
            print(f"Warning: Missing result for record {i}, using default values")
            curated_terms.append("UNKNOWN")
            confidences.append(0.0)

    # Add to dataframe
    df[f"curated_term_{suffix}"] = curated_terms
    df[f"confidence_{suffix}"] = confidences
    df[f"confident_{suffix}"] = df[f"confidence_{suffix}"] >= CONF_THRESHOLD

    # Save results
    output_file = f"curated_{suffix}.csv"
    df.to_csv(output_file, index=False)

    # Calculate accuracy
    accuracy, confident_count, total_count = calculate_accuracy(df, suffix)
    print(f"✅ Saved → {output_file} (≥{CONF_THRESHOLD}: {confident_count}/{total_count}, {accuracy:.2%})")

    # Download the file
    files.download(output_file)

    return df

def hybrid_curation_strategy(df, standard_terms, standard_embeddings, model, strategy="similarity_first"):
    """Two-stage hybrid curation strategy with configurable order"""

    if strategy == "similarity_first":
        # First stage: Curate all terms with similarity-guided approach
        print("\n==== STAGE 1: CONTEXT-ENRICHED SIMILARITY-GUIDED CURATION ====")
        df_first_stage = llm_curate_dataset(
            df.copy(),
            approach_type="similarity",
            standard_terms=standard_terms
        )

        # Identify low confidence terms
        low_conf_mask = df_first_stage["confidence_similarity"] < CONF_THRESHOLD
        low_conf_count = low_conf_mask.sum()

        stage1_suffix = "similarity"
        stage2_suffix = "std_terms"
        second_stage_name = "CONTEXT-ENRICHED STANDARD TERMS GUIDED"

    elif strategy == "standard_terms_first":
        # First stage: Curate all terms with standard terms approach
        print("\n==== STAGE 1: CONTEXT-ENRICHED STANDARD TERMS GUIDED CURATION ====")
        df_first_stage = llm_curate_dataset(
            df.copy(),
            approach_type="standard_terms",
            standard_terms=standard_terms
        )

        # Identify low confidence terms
        low_conf_mask = df_first_stage["confidence_std_terms"] < CONF_THRESHOLD
        low_conf_count = low_conf_mask.sum()

        stage1_suffix = "std_terms"
        stage2_suffix = "similarity"
        second_stage_name = "CONTEXT-ENRICHED SIMILARITY-GUIDED"

    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    # Report first stage results
    print(f"\nStage 1 results: {len(df_first_stage) - low_conf_count}/{len(df_first_stage)} terms curated with high confidence")
    print(f"Moving {low_conf_count} low-confidence terms to stage 2...")

    # Debug: Print a sample of low confidence terms
    if low_conf_count > 0:
        low_conf_df = df_first_stage[low_conf_mask]
        sample_count = min(5, len(low_conf_df))
        print(f"\nSample of low confidence terms being sent to stage 2 ({sample_count} of {low_conf_count}):")
        for i, (_, row) in enumerate(low_conf_df.head(sample_count).iterrows()):
            print(f"Term: {row['anatomy_uncurated']}")
            print(f"  → First stage mapping: {row[f'curated_term_{stage1_suffix}']} (confidence: {row[f'confidence_{stage1_suffix}']:.2f})")
            if i < sample_count - 1:  # Don't print separator after last item
                print()

    if low_conf_count == 0:
        print("No low-confidence terms to process in stage 2.")
        # Just rename the columns to match the hybrid naming
        strategy_suffix = "hybrid"
        df_final = df_first_stage.rename(columns={
            f"curated_term_{stage1_suffix}": f"curated_term_{strategy_suffix}",
            f"confidence_{stage1_suffix}": f"confidence_{strategy_suffix}",
            f"confident_{stage1_suffix}": f"confident_{strategy_suffix}"
        })

        # Calculate accuracy
        accuracy, confident_count, total_count = calculate_accuracy(df_final, strategy_suffix)

        # Save results
        output_file = f"curated_{strategy_suffix}.csv"
        df_final.to_csv(output_file, index=False)
        print(f"✅ Saved → {output_file} (≥{CONF_THRESHOLD}: {confident_count}/{total_count}, {accuracy:.2%})")
        files.download(output_file)

        return df_final

    # Extract only the low confidence rows for stage 2
    df_low_conf = df_first_stage[low_conf_mask].copy()

    # Second stage with opposite approach
    print(f"\n==== STAGE 2: {second_stage_name} CURATION (FOR LOW CONFIDENCE TERMS) ====")

    second_approach = "similarity" if stage2_suffix == "similarity" else "standard_terms"
    df_second_stage = llm_curate_dataset(
        df_low_conf,
        approach_type=second_approach,
        standard_terms=standard_terms
    )

    # Merge results: use second stage results for previously low-confidence terms
    df_final = df_first_stage.copy()
    df_final.loc[low_conf_mask, f"curated_term_{stage1_suffix}"] = df_second_stage[f"curated_term_{stage2_suffix}"]
    df_final.loc[low_conf_mask, f"confidence_{stage1_suffix}"] = df_second_stage[f"confidence_{stage2_suffix}"]
    df_final.loc[low_conf_mask, f"confident_{stage1_suffix}"] = df_second_stage[f"confident_{stage2_suffix}"]

    # Rename columns to reflect hybrid strategy
    strategy_suffix = "hybrid"
    df_final = df_final.rename(columns={
        f"curated_term_{stage1_suffix}": f"curated_term_{strategy_suffix}",
        f"confidence_{stage1_suffix}": f"confidence_{strategy_suffix}",
        f"confident_{stage1_suffix}": f"confident_{strategy_suffix}"
    })

    # Calculate accuracy
    accuracy, confident_count, total_count = calculate_accuracy(df_final, strategy_suffix)

    # Save results
    output_file = f"curated_{strategy_suffix}.csv"
    df_final.to_csv(output_file, index=False)
    print(f"✅ Saved → {output_file} (≥{CONF_THRESHOLD}: {confident_count}/{total_count}, {accuracy:.2%})")
    files.download(output_file)

    return df_final

def compare_approaches(similarity_df=None, std_terms_df=None, sim_std_df=None, std_sim_df=None):
    """Compare the accuracy of different approaches"""
    results = {}

    if similarity_df is not None:
        accuracy, confident_count, total_count = calculate_accuracy(similarity_df, "similarity")
        results["Context-enriched similarity-guided Curation"] = {"accuracy": accuracy, "confident": confident_count, "total": total_count}

    if std_terms_df is not None:
        accuracy, confident_count, total_count = calculate_accuracy(std_terms_df, "std_terms")
        results["Context-enriched Standard terms guided Curation"] = {"accuracy": accuracy, "confident": confident_count, "total": total_count}

    if sim_std_df is not None:
        accuracy, confident_count, total_count = calculate_accuracy(sim_std_df, "hybrid")
        results["Hybrid Context-enriched Curation (Similarity → Standard)"] = {"accuracy": accuracy, "confident": confident_count, "total": total_count}

    if std_sim_df is not None:
        accuracy, confident_count, total_count = calculate_accuracy(std_sim_df, "hybrid")
        results["Hybrid Context-enriched Curation (Standard → Similarity)"] = {"accuracy": accuracy, "confident": confident_count, "total": total_count}

    # Print comparison table
    print("\n===== ACCURACY COMPARISON =====")
    print(f"Confidence threshold: {CONF_THRESHOLD}")
    print("-" * 90)
    print(f"{'Approach':<55} | {'Accuracy':<10} | {'High Confidence':<15} | {'Total':<5}")
    print("-" * 90)

    for approach, data in results.items():
        print(f"{approach:<55} | {data['accuracy']:.2%} | {data['confident']:<15} | {data['total']:<5}")

    print("-" * 90)

    return results

# ================ MAIN EXECUTION =====================
def main():
    """Main pipeline execution flow with extended options"""
    print("========== FACEBASE ANATOMICAL TERM CURATION PIPELINE ==========")
    print("1. Single approach: Context-enriched similarity-guided Curation")
    print("2. Single approach: Context-enriched Standard terms guided Curation")
    print("3. Two-stage: Hybrid Context-enriched Curation (Similarity → Standard)")
    print("4. Two-stage: Hybrid Context-enriched Curation (Standard → Similarity)")
    print("5. Run all approaches and compare results")
    choice = input("Enter your choice (1-5): ")

    # Load standard embeddings
    standard_terms, standard_embeddings, model = load_or_create_standard_embeddings()

    # Upload uncurated CSV file
    print("\n==== LOADING UNCURATED DATA ====")
    print("Upload your uncurated CSV file...")
    uploaded = files.upload()
    uncurated_file = next(iter(uploaded))
    uncurated_df = pd.read_csv(uncurated_file)

    # Compute top matches for embedding-based approaches
    matched_df = compute_top_n_matches(
        uncurated_df,
        model,
        standard_terms,
        standard_embeddings
    )
    matched_df.to_csv("uncurated_with_matches.csv", index=False)
    print("✅ Saved → uncurated_with_matches.csv")

    # Initialize result DataFrames
    similarity_df = None
    std_terms_df = None
    sim_std_df = None
    std_sim_df = None

    if choice in ('1', '5'):
        print("\n==== RUNNING CONTEXT-ENRICHED SIMILARITY-GUIDED CURATION ====")
        similarity_df = llm_curate_dataset(
            matched_df.copy(),
            approach_type="similarity",
            standard_terms=standard_terms
        )

    if choice in ('2', '5'):
        print("\n==== RUNNING CONTEXT-ENRICHED STANDARD TERMS GUIDED CURATION ====")
        std_terms_df = llm_curate_dataset(
            matched_df.copy(),
            approach_type="standard_terms",
            standard_terms=standard_terms
        )

    if choice in ('3', '5'):
        print("\n==== RUNNING HYBRID CONTEXT-ENRICHED CURATION (SIMILARITY → STANDARD) ====")
        sim_std_df = hybrid_curation_strategy(
            matched_df.copy(),
            standard_terms=standard_terms,
            standard_embeddings=standard_embeddings,
            model=model,
            strategy="similarity_first"
        )

    if choice in ('4', '5'):
        print("\n==== RUNNING HYBRID CONTEXT-ENRICHED CURATION (STANDARD → SIMILARITY) ====")
        std_sim_df = hybrid_curation_strategy(
            matched_df.copy(),
            standard_terms=standard_terms,
            standard_embeddings=standard_embeddings,
            model=model,
            strategy="standard_terms_first"
        )

    if choice == '5':
        # Compare all approaches
        compare_approaches(
            similarity_df=similarity_df,
            std_terms_df=std_terms_df,
            sim_std_df=sim_std_df,
            std_sim_df=std_sim_df
        )

    print("\n✅ PIPELINE EXECUTION COMPLETE")

if __name__ == "__main__":
    main()

========== FACEBASE ANATOMICAL TERM CURATION PIPELINE ==========
1. Single approach: Context-enriched similarity-guided Curation
2. Single approach: Context-enriched Standard terms guided Curation
3. Two-stage: Hybrid Context-enriched Curation (Similarity → Standard)
4. Two-stage: Hybrid Context-enriched Curation (Standard → Similarity)
5. Run all approaches and compare results
Enter your choice (1-5): 5
Loading standard terms from cache: standard_terms_embeddings.pkl
Loaded 50 standard FaceBase terms.

==== LOADING UNCURATED DATA ====
Upload your uncurated CSV file...


Saving uncurated_highquality_100_rows_final.csv to uncurated_highquality_100_rows_final (3).csv


  0%|          | 0/100 [00:00<?, ?it/s]

Sample - Uncurated term: median palatal junction
  Top match: palate (similarity score: 0.6935)
Sample - Uncurated term: median palatal junction
  Top match: palate (similarity score: 0.6935)
Sample - Uncurated term: the maxillary palatine suture of the developing specimen
  Top match: maxillary palatine suture (similarity score: 0.8516)
Sample - Uncurated term: maxillary palatine suture, fibrous connective tissue region
  Top match: maxillary palatine suture (similarity score: 0.8328)
Sample - Uncurated term: lambdoid cranial joint
  Top match: lambdoid suture (similarity score: 0.8319)
Computed top 3 matches for 100 terms.
✅ Saved → uncurated_with_matches.csv

==== RUNNING CONTEXT-ENRICHED SIMILARITY-GUIDED CURATION ====


  0%|          | 0/25 [00:00<?, ?it/s]

...

<IPython.core.display.Javascript object>

✅ Saved → curated_hybrid.csv (≥0.8: 99/100, 99.00%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


===== ACCURACY COMPARISON =====
Confidence threshold: 0.8
------------------------------------------------------------------------------------------
Approach                                                | Accuracy   | High Confidence | Total
------------------------------------------------------------------------------------------
Context-enriched similarity-guided Curation             | 91.00% | 91              | 100  
Context-enriched Standard terms guided Curation         | 86.00% | 86              | 100  
Hybrid Context-enriched Curation (Similarity → Standard) | 97.00% | 97              | 100  
Hybrid Context-enriched Curation (Standard → Similarity) | 99.00% | 99              | 100  
------------------------------------------------------------------------------------------

✅ PIPELINE EXECUTION COMPLETE
